# Quantitative Momentum Strategy

In [3]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
import yfinance as yf
from datetime import datetime, timedelta

In [4]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
sp500 = sp500['Symbol'].to_list()

In [5]:
stocks_combined = ' '.join(sp500)

stocks_data = yf.Tickers(stocks_combined)
stocks_data = stocks_data.tickers

In [6]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)

for stock_ticker in stocks_data.values():

    response = stock_ticker.info

    if 'currentPrice' in response and '52WeekChange' in response:
        new_row = pd.Series(
            [response['symbol'], response['currentPrice'],(response['52WeekChange'] * 100), 'N/A'],
            index=my_columns
        )
    else:
        new_row = pd.Series(
            [response['symbol'], 0, 0, 'N/A'],
            index=my_columns
        )
    

    final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index = True)

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,125.24,42.34252,N/A
1,AOS,82.4,11.592639,N/A
2,ABT,111.31,2.63716,N/A
3,ABBV,189.29,25.916314,N/A
4,ACN,323.15,2.304745,N/A
...,...,...,...,...
498,XYL,130.15,21.680999,N/A
499,YUM,135.18,0.051808,N/A
500,ZBRA,322.3,29.839265,N/A
501,ZBH,110.36,-12.15474,N/A


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending= False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,477,VST,72.43,152.36933,N/A
1,350,NVDA,107.27,136.1891,N/A
2,222,GDDY,151.25,113.44905,N/A
3,348,NRG,70.0,86.567163,N/A
4,191,FICO,1600.38,85.8163,N/A
5,239,HWM,90.16,78.711593,N/A
6,44,ANET,319.34,78.13355,N/A
7,290,LLY,804.46,77.16261,N/A
8,430,SMCI,624.65,76.80942,N/A
9,279,KKR,108.62,75.73209,N/A


In [9]:
def getAmountToInvest():
    portfolio_size = input('Enter the Amount')
    val = 0.0
    while(True):
        try:
            val = float(portfolio_size)
            break
        except:
            portfolio_size = input('Enter the Amount Again')
    
    return (val)

In [11]:
x = getAmountToInvest() / len(final_dataframe.index)

for i in range(0 , len(final_dataframe.index)):
    final_dataframe.loc[i ,'Number of Shares to Buy'] = math.floor(x / final_dataframe.loc[i, 'Price'])

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,477,VST,72.43,152.36933,276
1,350,NVDA,107.27,136.1891,186
2,222,GDDY,151.25,113.44905,132
3,348,NRG,70.0,86.567163,285
4,191,FICO,1600.38,85.8163,12
5,239,HWM,90.16,78.711593,221
6,44,ANET,319.34,78.13355,62
7,290,LLY,804.46,77.16261,24
8,430,SMCI,624.65,76.80942,32
9,279,KKR,108.62,75.73209,184


## Building a Better Momentum Strategy

In [14]:
stocks_combined = ' '.join(sp500)

stocks_data = yf.Tickers(stocks_combined)
stocks_data = stocks_data.tickers

In [29]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One Year Price Return',
    'One Year Return Percentile',
    'Six Month Price Return',
    'Six Month Return Percentile',
    'Three Month Price Return',
    'Three Month Return Percentile',
    'One Month Price Return',
    'One Month Return Percentile',
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)


for stock_ticker in stocks_data.values():

    response = stock_ticker.info

    hist = stock_ticker.history(period="1y")

    if 'currentPrice' in response and not hist.empty:
        
        current_price = response['currentPrice']
        
        price_one_month = hist['Close'][-21]
        price_three_months = hist['Close'][-63]
        price_six_months = hist['Close'][-126]

        one_year_return = response['52WeekChange']
        six_months_return = ((current_price - price_six_months)/price_six_months)
        three_months_return = ((current_price - price_three_months)/price_three_months)
        one_month_return = ((current_price - price_one_month)/price_one_month)

        new_row = pd.Series(
            [response['symbol'], response['currentPrice'],'N/A',one_year_return,'N/A',six_months_return,'N/A',three_months_return,'N/A',one_month_return, 'N/A'],
            index=hqm_columns
        )
    else:
        new_row = pd.Series(
            [response['symbol'],0,'N/A',0,'N/A',0,'N/A',0,'N/A',0,'N/A'],
            index=hqm_columns
        )
    

    hqm_dataframe = pd.concat([hqm_dataframe, new_row.to_frame().T], ignore_index = True)

C:\Users\devag\AppData\Local\Temp\ipykernel_15728\3698694362.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price_one_month = hist['Close'][-21]
C:\Users\devag\AppData\Local\Temp\ipykernel_15728\3698694362.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price_three_months = hist['Close'][-63]
C:\Users\devag\AppData\Local\Temp\ipykernel_15728\3698694362.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price_six_months 

In [30]:
time_periods = [
                    'One Year',
                    'Six Month',
                    'Three Month',
                    'One Month'
               ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col],int(hqm_dataframe.loc[row, change_col]))

In [31]:
from statistics import mean

for row in hqm_dataframe.index:

    momentum_score = []

    for time_period in time_periods:
        momentum_score.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])

    hqm_dataframe.loc[row, 'Score'] = mean(momentum_score)

In [38]:
hqm_dataframe.sort_values('Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)

In [39]:
x = getAmountToInvest() / 50
for i in range(0 , len(hqm_dataframe.index)):
    hqm_dataframe.loc[i ,'Number of Shares to Buy'] = math.floor(x / hqm_dataframe.loc[i, 'Price'])

In [40]:
writer = pd.ExcelWriter('Momentum Strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy.xlsx', index=False)

In [41]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [42]:
columns_format = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One Year Price Return', percent_template],
    'E': ['One Year Return Percentile', percent_template],
    'F': ['Six Month Price Return', percent_template],
    'G': ['Six Month Return Percentile', percent_template],
    'H': ['Three Month Price Return', percent_template],
    'I': ['Three Month Return Percentile', percent_template],
    'J': ['One Month Price Return', percent_template],
    'K': ['One Month Return Percentile', percent_template],
    'L': ['Score', percent_template]
}

for column in columns_format.keys():
    writer.sheets["Momentum Strategy.xlsx"].set_column(f'{column}:{column}', 25, columns_format[column][1])
    writer.sheets["Momentum Strategy.xlsx"].write(f'{column}1', columns_format[column][0], columns_format[column][1])

writer.close()
